# Chains

## simple chain


In [4]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
load_dotenv()
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

prompt_template = PromptTemplate(template="generate 5 intresting point about {topic}",input_variables=['topic'])

chain = prompt_template|model|parser

res = chain.invoke({'topic':"protein powder"})

print(res)

1. **Protein powder isn't just for bodybuilders:** While often associated with muscle growth, protein powder can benefit a wide range of individuals, including older adults needing help maintaining muscle mass, athletes recovering from intense training, or individuals looking to increase their overall protein intake for better satiety and weight management.  It's a versatile supplement, not a niche product.

2. **The source matters significantly:**  Different protein powders offer different amino acid profiles, digestibility, and potential health benefits. Whey protein is fast-digesting and ideal post-workout, casein protein is slow-digesting and good before bed, while plant-based options like soy, pea, or brown rice protein offer alternatives for those with dairy sensitivities or vegan lifestyles.  Understanding the source helps tailor the powder to individual needs.

3. **Beyond muscle building, it impacts other areas of health:** Protein contributes to numerous bodily functions beyo

In [6]:

chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


# parallel Chain

In [12]:
from langchain_core.runnables import RunnableParallel
import streamlit as st
model1 = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
model2 = ChatGoogleGenerativeAI(model='gemini-1.5-pro')

prompt_template1 = PromptTemplate(
    template="generate notes on given topic {topic}",
    input_variables=['topic']
)

prompt_template2 = PromptTemplate(
    template="generate 5-6 question and anwser on topic {topic}",
    input_variables=['topic']
)

prompt_template3 = PromptTemplate(
    template="merge notes and questions into single document notes->{notes}, questions->{quiz}",
    input_variables=['notes','quiz']
)

parallel_chain = RunnableParallel({
    'notes': prompt_template1|model1|parser,
    'quiz':prompt_template2|model2|parser
})

third_chain = prompt_template3|model1|parser

final_chain = parallel_chain|third_chain

text = """ Linear-regression models are relatively simple and provide an easy-to-interpret mathematical formula that can generate predictions. Linear regression can be applied to various areas in business and academic study.

You’ll find that linear regression is used in everything from biological, behavioral, environmental and social sciences to business. Linear-regression models have become a proven way to scientifically and reliably predict the future. Because linear regression is a long-established statistical procedure, the properties of linear-regression models are well understood and can be trained very quickly."""



res = final_chain.invoke({'topic':text})
print(res)



## Linear Regression Model Notes and Questions

**I. Core Concept:**

* Simple, easily interpretable mathematical formula for prediction.
* Predicts a continuous dependent variable based on one or more independent variables.
* Assumes a linear relationship between variables.

**II. Applicability:**

* **Broad range of fields:** Biological, behavioral, environmental, social sciences, and business.
* **Versatile prediction tool:** Used for forecasting and understanding relationships between variables.  Examples include predicting sales based on advertising spend, or crop yield based on rainfall.  Another example is predicting real estate prices based on features like size, location, and age of the property.

**III. Advantages:**

* **Simplicity & Interpretability:** The model's equation directly shows the impact of each independent variable on the dependent variable. Coefficients are easily understood (as seen in Q2).
* **Established Statistical Procedure:** Well-understood properties, m

In [10]:
final_chain.get_graph().print_ascii()

                    +---------------------------+                      
                    | Parallel<notes,quiz>Input |                      
                    +---------------------------+                      
                       ***                   ***                       
                   ****                         ****                   
                 **                                 **                 
    +----------------+                          +----------------+     
    | PromptTemplate |                          | PromptTemplate |     
    +----------------+                          +----------------+     
             *                                           *             
             *                                           *             
             *                                           *             
+------------------------+                  +------------------------+ 
| ChatGoogleGenerativeAI |                  | ChatGoogleGenerati